In [12]:
from __future__ import print_function, division

# import sys
# sys.path.append(root_folder)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pathlib

from torch.autograd import Variable


from models.convengers import *
from models.solver import NickFury
from utils import dictionary
from utils import visualizations

In [2]:
dataset_folder = "data/tiny-imagenet-200/"
data_dir = pathlib.Path(dataset_folder)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
image_datasets = {x: datasets.ImageFolder(data_dir / x, data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=20, shuffle=True, pin_memory=True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
chpt = torch.load("latest.pt")

In [5]:
print(chpt.keys())

dict_keys(['net'])


In [16]:
model = torchvision.models.resnet101(pretrained=True)

In [17]:
inputs, classes = next(iter(dataloaders['train']))
class_preds = model(inputs)
print(classes)
print(torch.argmax(class_preds,dim=1))

tensor([ 21, 159, 119, 167,  15, 193,  27, 123,   6,  85,  68, 103, 111, 137,
         44, 159,  22,  25,  69,  65])
tensor([267, 601, 459, 446, 348, 616, 979,  88, 220, 797, 430, 975, 852, 987,
        323, 673, 570, 220, 721, 569])


In [59]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [60]:
adversary_classes = torch.zeros(classes.shape,dtype=torch.long)
input_images = Variable(inputs, requires_grad=True)

In [61]:
model_scores = model(input_images)
criterion =  nn.CrossEntropyLoss()
loss = criterion(model_scores, adversary_classes)
loss.backward()

image_grad = input_images.grad.data

In [62]:
for epsilon in torch.arange(0,1,0.1):
    adversaries = fgsm_attack(input_images, epsilon, image_grad)
    adverary_scores = model(adversaries)
    print(epsilon)
    print(torch.argmax(adverary_scores,dim=1))
    print()

tensor(0.)
tensor([182,  24, 167, 110, 144, 119,  20, 126, 159, 116,  81,   5, 174, 146,
         89,  95, 155, 191,  44, 161])

tensor(0.1000)
tensor([161,  37, 109,  29,  90, 133,  20,  33,  85,  89, 154,  10,   0, 146,
        136,  95,  81, 191,  44,   0])

tensor(0.2000)
tensor([161, 143, 109,   0,  90, 147,  20, 136,  30,   4, 157,  26,  47,  89,
        134, 173, 127, 191, 159, 199])

tensor(0.3000)
tensor([161, 143, 139,   0,  90, 142,  20, 190, 149,   5, 157, 152,   0, 146,
        113,  95, 127, 191,   0,  14])

tensor(0.4000)
tensor([188, 143, 117, 197, 105, 133,  20, 164, 149,  89, 133,  14,   0, 110,
         65, 139, 127, 181,  41, 185])

tensor(0.5000)
tensor([185, 143,  72,  53, 149, 133,  20, 159, 149,  80, 119, 196,   0, 146,
        113, 139, 127, 191,  36,  25])

tensor(0.6000)
tensor([ 83, 143, 139,  22, 149, 167,  22,  59, 149, 159, 112,   4,   0, 146,
        113,  95, 127, 191,  37, 196])

tensor(0.7000)
tensor([185, 143, 119,  55, 120, 129,  20, 199,  16,  63, 